In [46]:
import pandas as pd
import numpy as np
from datetime import date
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [47]:
pv = 1000
r = 0.1
t = 4

In [48]:
af = ((1/r)-(1/(r*(1+r)**t))); af

3.169865446349295

In [49]:
pmt = pv/af; pmt

315.4708037060976

Basic facts from the loan

In [50]:
r = 0.1
years = 4
payments_year = 1
t = years*payments_year
pv = 1000
t_0 = date(2018,10,21)

how much is the value of each payment (pmt)

In [51]:
pmt = np.pmt(r,t,pv); pmt

-315.4708037060977

Now, how much of the pmt is interests (i) and how much is principal (p)?

In [52]:
# Period to calculate
per = 1

# Calculate the interest
ipmt = np.ipmt(r/payments_year, per, t, pv); ipmt

array(-100.)

In [53]:
# Calculate the principal
ppmt = np.ppmt(r/payments_year, per, t, pv); ppmt

-215.47080370609768

Now, let us create a table for this data set.
First we create the date range for all the payment periods

In [54]:
rng = pd.date_range(start=t_0,periods=t,freq='365D')
rng.name = "Date"

Then, we create the columns for the table. We do that by creating a "Data Frame" in Pandas
we use the date range as index.

In [55]:
df = pd.DataFrame(index=rng, 
                  columns=['Beg Balance',
                           'intermed',
                           'Interest', 
                           'Total year end payment',
                           'Amortization of loan', 
                           'End of year Balance'], 
                            dtype='float')
df.reset_index(inplace=True)
df.index += 1
df.index.name = "Year"

In [56]:
df.head()

,Date,Beg Balance,intermed,Interest,Total year end payment,Amortization of loan,End of year Balance
Year,,,,,,,
1,2018-10-21,nan,nan,nan,nan,nan,nan
2,2019-10-21,nan,nan,nan,nan,nan,nan
3,2020-10-20,nan,nan,nan,nan,nan,nan
4,2021-10-20,nan,nan,nan,nan,nan,nan


now we populate the table with the values from the pmts

In [57]:
df["Total year end payment"] = np.pmt(r/payments_year, t, pv)
df["Amortization of loan"] = np.ppmt(r/payments_year, df.index, t, pv)
df["Interest"] = np.ipmt(r/payments_year, df.index, t, pv)
df["End of year Balance"] = (df["Amortization of loan"]).cumsum()
df["intermed"] = pv
df["Beg Balance"] = (df["End of year Balance"]+ df["intermed"])

# I am stuck. .....
# Whenever I try to calculate the value of the Beginning of year balance. I end up with a science notation number.
#  df["Beg Balance"] = (df["End of year Balance"]+ df["intermed"])
# Basically, it should calculate the pv - the value in the "End of Year Balance" column...

df.head()


,Date,Beg Balance,intermed,Interest,Total year end payment,Amortization of loan,End of year Balance
Year,,,,,,,
1,2018-10-21,784.529,1000,-100.000,-315.471,-215.471,-215.471
2,2019-10-21,547.511,1000,-78.453,-315.471,-237.018,-452.489
3,2020-10-20,286.792,1000,-54.751,-315.471,-260.720,-713.208
4,2021-10-20,0.000,1000,-28.679,-315.471,-286.792,-1000.000
